<a href="https://colab.research.google.com/github/Imranzr/EEG_Based_Auditory_Brain_Computer_Interface_Using_Deep_Learning_and_Machine_Learning_Model/blob/main/Auditory_BCI_Dynamical_Window_Epoching_based_Machine_learning(ML)_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.6 MB/s eta 0:00:00


In [3]:
pip install pyriemann

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 5.4 MB/s eta 0:00:00


In [10]:
import os
import argparse
import sys
import mne
import math
import time
import json
import numpy as np
from scipy.signal import butter, filtfilt
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef

start = time.time()

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True
        elif shell == 'TerminalInteractiveShell':
            return False
        else:
            return False
    except NameError:
        return False

if is_notebook():

    args = argparse.Namespace(s=None, c=None)
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('-s', default=None)
    parser.add_argument('-c', default=None, type=int)
    args = parser.parse_args(args=[])

print(args.s)
print(args.c)
print(__doc__)

subject = 'sub-B'
if args.s is not None:
    subject = args.s
test_class =1
if args.c is not None:
    test_class = args.c

import numpy as np
fnum = np.array([[1,4],
                 [2,5],
                 [3,6]])

trig_id = [2,8,32]
tasks = ['low', 'low', 'mid', 'mid', 'high', 'high']
reject={'eeg':100e-6,'eog':500e-6}

import os
import json
repository_base = os.path.dirname(os.path.dirname(os.path.abspath('/content/drive/MyDrive/dataverse_files/results/sub-B_classification_scores.json')))

#repository_base = os.path.dirname(os.path.dirname(os.path.abspath('file_path')))
base = os.path.join(repository_base, "eeg")
save_base = os.path.join(repository_base, "results")
if not os.path.exists(save_base):
    os.makedirs(save_base)

Fs = 1000
fc = [1, 40]
resample = None

from scipy.signal import butter, filtfilt
def apply_filter(data, b, a):
    r = filtfilt(b=b, a=a, x=data)
    return r
b,a = butter(N = 2, Wn = np.array(fc)/(Fs/2), btype = 'bandpass', output = 'ba')

t_file = []
nt_file = []

target_file = []
non_target_file = []

for i in range(len(fnum.ravel())):
    fname = os.path.join(base, subject, "eeg", "%s_task-%s_run-%d_eeg.vhdr" % (subject, tasks[i], fnum.ravel()[i]))
    print(fname)
    if np.any(fnum[test_class-1] == fnum.ravel()[i]):
        if isinstance(target_file, list):
            target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            target_file = target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            t_file.append(fnum.ravel()[i])
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            target_file = mne.concatenate_raws([target_file, tmp])
            t_file.append(fnum.ravel()[i])
    else:
        if isinstance(non_target_file, list):
            non_target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            non_target_file = non_target_file.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            nt_file.append(fnum.ravel()[i])
        else:
            tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
            tmp = tmp.apply_function(apply_filter, channel_wise=True, b=b, a=a)
            non_target_file = mne.concatenate_raws([non_target_file, tmp])
            nt_file.append(fnum.ravel()[i])
if resample is not None:
    target_file.resample(resample)
    non_target_file.resample(resample)

if resample != None:
    target_file.resample(resample)
    non_target_file.resample(resample)

target_eve = mne.events_from_annotations(target_file)
non_target_eve = mne.events_from_annotations(non_target_file)

event_id={'target_stimulus_id':-100,'non_target_stimulus_id':-500}

target_eve = mne.merge_events(target_eve[0], [trig_id[test_class-1]], event_id['target_stimulus_id'], replace_events=True)
non_target_eve = mne.merge_events(non_target_eve[0], [trig_id[test_class-1]], event_id['non_target_stimulus_id'], replace_events=True)

tmin,tmax= -0.02, 1.4
baseline=(0.0,0.01)
epochs_target = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'],
                            tmin=tmin, tmax=tmax, baseline=baseline,
                           reject=reject, preload=True)

for idx, event in enumerate(epochs_target.events):
    if event[2] == -100:
        annot_time = event[0] / epochs_target.info['sfreq']
        epochs_target.annotations.append(
            onset=annot_time, duration=0.001, description="target_event -100"
        )


None
None
Automatically created module for IPython interactive environment
/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-1_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-1_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 317919  =      0.000 ...   317.919 secs...


<ipython-input-10-2d7a8e5e8932>:91: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-10-2d7a8e5e8932>:91: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  target_file = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-4_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-low_run-4_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 312879  =      0.000 ...   312.879 secs...


<ipython-input-10-2d7a8e5e8932>:95: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-10-2d7a8e5e8932>:95: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-mid_run-2_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-mid_run-2_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 314879  =      0.000 ...   314.879 secs...


...

/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-3_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-3_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 319199  =      0.000 ...   319.199 secs...


<ipython-input-10-2d7a8e5e8932>:105: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-10-2d7a8e5e8932>:105: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


/content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-6_eeg.vhdr
Extracting parameters from /content/drive/MyDrive/dataverse_files/eeg/sub-B/eeg/sub-B_task-high_run-6_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 321519  =      0.000 ...   321.519 secs...


<ipython-input-10-2d7a8e5e8932>:105: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))
<ipython-input-10-2d7a8e5e8932>:105: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (100.00 Hz) will be stored.
  tmp = mne.io.read_raw_brainvision(fname, preload=True, eog=('hEOG', 'vEOG'))


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S129', 'Stimulus/S132', 'Stimulus/S144']
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S129', 'Stimulus/S132', 'Stimulus/S144']
Not setting metadata
119 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 119 events and 1421 original time points ...
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF8']
    Rejecting  epo

# Dynamical_Window_Epoching

In [13]:
latencies_p300 = []
target_epochs_dynamic = []
non_target_epochs_dynamic = []
for epoch_idx, epoch in enumerate(epochs_target):
    picks = mne.pick_types(epochs_target.info, eeg=True, exclude='bads')
    data = epoch[picks].mean(axis=0)
    times = epochs_target.times
    p300_time_window = (0.3, 1.4)
    idx_start = np.searchsorted(times, p300_time_window[0])
    idx_end = np.searchsorted(times, p300_time_window[1])
    if idx_end > idx_start:
        data_window = data[idx_start:idx_end]
        peak_idx = np.argmax(data_window)
        peak_time = times[idx_start + peak_idx]
        latency_p300 = peak_time
    else:
        latency_p300 = None
    latencies_p300.append(latency_p300)
    if latency_p300 is not None:
        print(f"Epoch {epoch_idx + 1}: P300 latency = {latency_p300:.3f} seconds")
    else:
        print(f"Epoch {epoch_idx + 1}: P300 latency not found")
    if latency_p300 is not None:
        tmin_dynamic = max(-0.38, latency_p300 - 0.33)
        tmax_dynamic = latency_p300 + 0.33
        baseline_start = max(tmin_dynamic, baseline[0])
        baseline_end = min(tmax_dynamic, baseline[1])
        if baseline_start >= baseline_end:
            print(f"Adjusting baseline to fit within the epoch window: [{tmin_dynamic}, {tmax_dynamic}] s")
            baseline_start = tmin_dynamic
            baseline_end = tmin_dynamic + 0.045
        target_epoch_dynamic = mne.Epochs(target_file, events=target_eve, event_id=event_id['target_stimulus_id'],
                                          tmin=tmin_dynamic, tmax=tmax_dynamic,
                                          baseline=(baseline_start, baseline_end),
                                          reject=reject, preload=True)
        non_target_epoch_dynamic = mne.Epochs(non_target_file, events=non_target_eve, event_id=event_id['non_target_stimulus_id'],
                                              tmin=tmin_dynamic, tmax=tmax_dynamic,
                                              baseline=(baseline_start, baseline_end),
                                              reject=reject, preload=True)
        target_epochs_dynamic.append(target_epoch_dynamic)
        non_target_epochs_dynamic.append(non_target_epoch_dynamic)
    else:
        print("P300 latency not found, using default epoch window of -0.1 to 0.5 sec")
        target_epoch_default = mne.Epochs(target, events=target_eve, event_id=event_id['target_stimulus_id'],
                                          tmin=tmin, tmax=tmax, baseline=baseline,
                                          reject=reject, preload=True)
        non_target_epoch_default = mne.Epochs(non_target_file, events=non_target_eve, event_id=event_id['non_target_stimulus_id'],
                                              tmin=tmin, tmax=tmax, baseline=baseline,
                                              reject=reject, preload=True)
        target_epochs_dynamic.append(target_epoch_default)
        non_target_epochs_dynamic.append(non_target_epoch_default)

print("P300 latencies (in seconds) for all epochs:", latencies_p300)


Streaming output truncated to the last 5000 lines.
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F6', 'F8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F5', 'F6', 'F8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F6', 'F8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F6']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F6', 'F8']
    Rejecting  epoch based on EEG : ['Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp2', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2', 'AF7', 'AF4', 'AF8

In [15]:
all_epochs = mne.concatenate_epochs([target_epoch_dynamic, non_target_epoch_dynamic])
epochs = all_epochs.copy().pick_types(eeg=True, eog=False)

Not setting metadata
251 matching events found
Applying baseline correction (mode: mean)


<ipython-input-15-017772bcdbcf>:1: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs([target_epoch_dynamic, non_target_epoch_dynamic])


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


# Logistic Regression

In [16]:


clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    LogisticRegression( penalty='l1', solver='liblinear', multi_class='ovr'))
epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    preds[test_idx] = clf.predict(epochs_data[test_idx])

report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was depre

{'non-target': {'precision': 0.7971014492753623, 'recall': 0.953757225433526, 'f1-score': 0.868421052631579, 'support': 173.0}, 'target': {'precision': 0.8181818181818182, 'recall': 0.46153846153846156, 'f1-score': 0.5901639344262295, 'support': 78.0}, 'accuracy': 0.8007968127490039, 'macro avg': {'precision': 0.8076416337285903, 'recall': 0.7076478434859939, 'f1-score': 0.7292924935289042, 'support': 251.0}, 'weighted avg': {'precision': 0.8036523208877271, 'recall': 0.8007968127490039, 'f1-score': 0.7819507131095977, 'support': 251.0}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


# Support Vector Machine (SVM)

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    SVC(kernel='linear', C=1, decision_function_shape='ovr'))


epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []
preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds
report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)




{'non-target': {'precision': 0.839572192513369, 'recall': 0.9075144508670521, 'f1-score': 0.8722222222222222, 'support': 173.0}, 'target': {'precision': 0.75, 'recall': 0.6153846153846154, 'f1-score': 0.676056338028169, 'support': 78.0}, 'accuracy': 0.8167330677290837, 'macro avg': {'precision': 0.7947860962566845, 'recall': 0.7614495331258337, 'f1-score': 0.7741392801251956, 'support': 251.0}, 'weighted avg': {'precision': 0.8117370091825212, 'recall': 0.8167330677290837, 'f1-score': 0.8112623060184926, 'support': 251.0}}


# Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    RandomForestClassifier(n_estimators=100, random_state=42))

epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):

    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds

report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)




{'non-target': {'precision': 0.7066115702479339, 'recall': 0.9884393063583815, 'f1-score': 0.8240963855421687, 'support': 173.0}, 'target': {'precision': 0.7777777777777778, 'recall': 0.08974358974358974, 'f1-score': 0.16091954022988506, 'support': 78.0}, 'accuracy': 0.7091633466135459, 'macro avg': {'precision': 0.7421946740128558, 'recall': 0.5390914480509856, 'f1-score': 0.4925079628860269, 'support': 251.0}, 'weighted avg': {'precision': 0.7287269654165706, 'recall': 0.7091633466135459, 'f1-score': 0.6180095571184312, 'support': 251.0}}


# Polynomial Logistic Regression

In [19]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

clf = make_pipeline(XdawnCovariances(8),
                    TangentSpace(metric='logeuclid'),
                    PolynomialFeatures(degree=2, include_bias=False),
                    LogisticRegression(penalty='l2', solver='liblinear', multi_class='ovr'))

epochs_data = epochs.get_data()
labels = epochs.events[:, -1]
preds = np.zeros(len(labels))

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

preds = np.empty(len(labels))
for train_idx, test_idx in cv.split(epochs_data, labels):
    clf.fit(epochs_data[train_idx], labels[train_idx])
    train_preds = clf.predict(epochs_data[train_idx])
    test_preds = clf.predict(epochs_data[test_idx])
    train_accuracy = accuracy_score(labels[train_idx], train_preds)
    test_accuracy = accuracy_score(labels[test_idx], test_preds)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_loss = np.mean(train_preds != labels[train_idx])
    test_loss = np.mean(test_preds != labels[test_idx])
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    preds[test_idx] = test_preds


report = classification_report(labels, preds, target_names=['non-target', 'target'], output_dict=True)
print(report)




/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was depre

{'non-target': {'precision': 0.7347826086956522, 'recall': 0.976878612716763, 'f1-score': 0.8387096774193549, 'support': 173.0}, 'target': {'precision': 0.8095238095238095, 'recall': 0.21794871794871795, 'f1-score': 0.3434343434343434, 'support': 78.0}, 'accuracy': 0.7410358565737052, 'macro avg': {'precision': 0.7721532091097308, 'recall': 0.5974136653327404, 'f1-score': 0.5910720104268492, 'support': 251.0}, 'weighted avg': {'precision': 0.7580089579569919, 'recall': 0.7410358565737052, 'f1-score': 0.6847994142686341, 'support': 251.0}}


In [ ]:
#SVM_plt
epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()


for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()

for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

plt.tight_layout()
plt.show()
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

#plt_Random_forest
epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()


for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()

for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

plt.tight_layout()
plt.show()

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

#plt_P_L_R
epochs_range = range(1, len(train_accuracies) + 1)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
ax1.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='x')
ax1.set_title('Train and Test Accuracy')
ax1.set_xlabel('Fold')
ax1.set_ylabel('Accuracy')
ax1.legend()

for i in range(len(train_accuracies)):
    ax1.annotate(f'{train_accuracies[i]:.2f}', (epochs_range[i], train_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax1.annotate(f'{test_accuracies[i]:.2f}', (epochs_range[i], test_accuracies[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
ax2.plot(epochs_range, train_losses, label='Train Loss', marker='o')
ax2.plot(epochs_range, test_losses, label='Test Loss', marker='x')
ax2.set_title('Train and Test Loss')
ax2.set_xlabel('Fold')
ax2.set_ylabel('Loss')
ax2.legend()


for i in range(len(train_losses)):
    ax2.annotate(f'{train_losses[i]:.2f}', (epochs_range[i], train_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
    ax2.annotate(f'{test_losses[i]:.2f}', (epochs_range[i], test_losses[i]),
                 textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)
plt.tight_layout()
plt.show()


cm = confusion_matrix(labels, preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non-target', 'target'], yticklabels=['non-target', 'target'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()